In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def phases(iq: np.ndarray,start:int):
    list = []
    for i in range(start,len(iq)-2,6):
        list.append(
            np.angle(complex(iq[i],iq[i+1]))
        )
    return np.array(list)



antenna_conf= "2-2-0-5-6"

data = pd.read_json("../folder/data.json")
data = data[data["Switch Order"] == antenna_conf]
#data = data[data["TAG"] != "B"]
#data = data[data["rssi"] >-70]
del data["level_0"]
del data["index"]
data=data.reset_index()
data.iloc[0]["iq"]


[-142,
 -137,
 120,
 -126,
 126,
 139,
 -167,
 104,
 -91,
 -169,
 154,
 -86,
 87,
 180,
 -185,
 84,
 -349,
 -6,
 -266,
 -275,
 -302,
 -228,
 326,
 98,
 107,
 377,
 197,
 336,
 -229,
 -245,
 34,
 -386,
 -54,
 -394,
 144,
 308,
 -178,
 348,
 -138,
 370,
 5,
 -343,
 286,
 -243,
 262,
 -277,
 -141,
 302,
 -352,
 122,
 -315,
 183,
 228,
 -244,
 379,
 39,
 383,
 -31,
 -307,
 122,
 -341,
 -187,
 -369,
 -111,
 337,
 -3,
 238,
 297,
 267,
 272,
 -309,
 -171,
 -129,
 -378,
 -161,
 -355,
 243,
 240,
 -19,
 379,
 28,
 389,
 -112,
 -307]

In [25]:
tdata = []
tlabels = []

for _, row in data.iterrows():
    for i in range(16,len(row["iq"])-6,3*2):
        tdata.append(row["iq"][i:i+6])
        tlabels.append(row["Angle"])


# On IQ samples

In [34]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#Loading the Iris dataset

label_mapper = lambda x: {"0":0,
            "30":1,
                "60":2,
                "90":3,
                "120":4,
                "150":5,
                "180":6,
                }.get(str(x))

X, y = tdata, [label_mapper(x) for x in tlabels]

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True)

#Training the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [35]:
#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names={"0","30","60","90","120","150","180"}))

Accuracy: 0.9709441611892622

Classification Report:
              precision    recall  f1-score   support

         120       0.97      0.96      0.97      4652
          60       0.97      0.98      0.97      4521
         150       0.99      0.99      0.99      4823
          30       0.99      0.98      0.98      4663
           0       0.96      0.95      0.95      4640
         180       0.95      0.97      0.96      4570
          90       0.97      0.97      0.97      4689

    accuracy                           0.97     32558
   macro avg       0.97      0.97      0.97     32558
weighted avg       0.97      0.97      0.97     32558



In [41]:
antenna_conf= "14-14-13-8-10"

data = pd.read_json("../folder/data.json")
data = data[data["Switch Order"] == antenna_conf]

vdata = []
vlabels = []

for _, row in data.iterrows():
    for i in range(16,len(row["iq"])-6,3*2):
        vdata.append(row["iq"][i:i+6])
        vlabels.append(row["Angle"])

vX, vy = vdata, [label_mapper(x) for x in vlabels]

#Making predictions on the test set
predictions = model.predict(vX)

#Calculating accuracy
accuracy = accuracy_score(vy, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(vy, predictions, target_names={"0","30","60","90","120","150","180"}))


Accuracy: 0.5106040617254337

Classification Report:
              precision    recall  f1-score   support

         120       0.24      0.29      0.26     12716
          60       0.56      0.54      0.55     12672
         150       0.55      0.50      0.52     14113
          30       0.77      0.73      0.75     15345
           0       0.35      0.34      0.34     15466
         180       0.58      0.74      0.65     14256
          90       0.53      0.43      0.47     15488

    accuracy                           0.51    100056
   macro avg       0.51      0.51      0.51    100056
weighted avg       0.52      0.51      0.51    100056



# Real Time

In [3]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import datetime


# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
def on_message(client, userdata, msg):
    data = json.loads(msg.payload)
    prediction = model.predict([data[1]["aoa"]["iq"]])
    print({ "0":0,
            "1":30,
            "2":60,
            "3":90,
            "4":120,
            "5":150,
            "6":180}.get(str(prediction[0])))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)

now = datetime.datetime.now()
client.loop_forever()


Conectado com o código de resultado 0


KeyboardInterrupt: 

16
22
28
34
40
